In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [2]:
conn = sqlite3.connect("D:/News/news.db")

conn2 = sqlite3.connect("D:/News/price.db")

cot=pd.read_sql('SELECT * FROM COT', conn)

sym_list=['EUR', 'GBP', 'NZD' ,'AUD', 'CAD', 'JPY', 'CHF']
pair_dict={'EUR':'EURUSD', 'GBP':'GBPUSD', 'NZD':'NZDUSD', 'AUD':'AUDUSD', 'CAD': 'USDCAD', 'JPY': 'USDJPY', 'CHF': 'USDCHF'}

In [3]:
for s in sym_list[:1]:
    product=cot[cot['PRODUCT']==s]

    price = pd.read_sql("Select * from PRICE where Sym='{}' and date(Date)>='2018'".format(pair_dict[s]),conn2)

    price.drop(['Sym'], axis=1, inplace=True)

    price['Date Time'] = price.Date + ' ' + price.Time

    price['Date Time']=pd.to_datetime(price['Date Time'])

    price.set_index('Date Time', inplace=True)

    daily_price = price['Close'].resample('5D').agg({'Close':'last'})

    product.set_index(pd.to_datetime(product.DATE), inplace=True)

    product['max_comm']=product['Net Comm'].rolling(150).max()
    product['min_comm']=product['Net Comm'].rolling(150).min()

    product['CX']=(product['Net Comm']-product.min_comm)/(product.max_comm-product.min_comm)

    product.dropna(inplace=True)

    merged=product.join(daily_price).dropna()

    fig, ax = plt.subplots(3,1, figsize=(30,15), gridspec_kw={'height_ratios': [2, 1, 1]})
    ax[0].plot(merged.Close.iloc[-100:])
    ax[0].set_title(s, fontdict={'fontsize':24})
    ax[1].plot(merged.CX.iloc[-100:])
    ax[1].axhline(0.75, color='red')
    ax[1].set_title('CX', fontdict={'fontsize':24})
    ax[1].axhline(0.25, color='red')
    ax[2].plot(merged.OI.iloc[-100:])
    ax[2].set_title('OI', fontdict={'fontsize':24})

    plt.savefig('report/{}.jpg'.format(s))
    plt.close()

C:\Users\zekec\AppData\Local\Temp\ipykernel_9580\1214437652.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product['max_comm']=product['Net Comm'].rolling(150).max()
C:\Users\zekec\AppData\Local\Temp\ipykernel_9580\1214437652.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product['min_comm']=product['Net Comm'].rolling(150).min()
C:\Users\zekec\AppData\Local\Temp\ipykernel_9580\1214437652.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using